## 병렬화하기
- Optuna Optimization을 수동으로 하려면
    1. RDB 서버 켜기(예제에선 MySQL)
    2. `-storage` 인자를 넣어 `study`를 만듦
    3. `study`를 여러 노드, 프로세스에 나눔
    
- 위 과정은 [쿠버네티스 예제](https://github.com/optuna/optuna-examples/tree/main/kubernetes)로도 할 수 있음

- python cmd 창에서..
```
mysql -u root -e "CREATE DATABASE IF NOT EXISTS example"
optuna create-study --study-name 'distributed-example' --storage 'mysql://root@localhost/example'
```

- 이 다음 Optimization Script를 짬

- 파일명 : `foo.py`
```python
import optuna

def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x-2) ** 2

if __name__ == "__main__":
    study = optuna.load_study(
        study_name = 'distribute-example', storage = 'mysql://root@localhost/example')
    study.optimize(objective, n_trials = 100)
```

- 이후는 여러 터미널을 띄우고 각 터미널에 `python foo.py`를 입력하면 병렬로 처리되는 걸 볼 수 있음

#### 참고사항
1. `n_trials`는 각 프로세스(즉 각 터미널)에서 실행되는 `trial`의 숫자임. 즉 2개의 터미널을 켜고 `n_trials = 100`일 경우 총 200 `trials`이 되는 것임
    - 전체 수를 보려면 `optuna.study.MaxTrialsCallback`이 있다
    
2. SQLite는 성능 상의 이유로 추천하지 않으며, `MySQL`이나 `PostgreSQL`을 추천한다.